In [1]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
!pip install hazm
!pip install fasttext
from hazm import *
import fasttext.util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
  device = torch.device("cpu")
  print("GPU not available, CPU used")

GPU is available


In [3]:
import re

from hazm import *
import re
normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def remove_emojis_punctuations(text):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = re.sub(emoj, '', text)
    punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،؟'
    text = text.translate(str.maketrans(punctuations, ' '*len(punctuations)))
    from string import digits
    digits = digits + "۰۱۲۳۴۵۶۷۸۹"
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    return text

def preprocess_text(text):
  return remove_emojis_punctuations(text)
def preprocess_data_frame(data_frame, column="sent"):
  for index, row in data_frame.iterrows():
      row[column] = remove_emojis_punctuations(row[column])
      row[column] = normalizer.normalize(row[column])
      for sent in sent_tokenize(row[column]):
        for word in word_tokenize(row[column]):
          if re.search(r'[a-zA-Z]', word):
            row[column] = row[column].replace(word, "")
          else:
            new_word = lemmatizer.lemmatize(word)
            if new_word.__contains__("#"):
              new_word = new_word.split("#")[0]
            
            new_word = new_word.replace("\u200c"," ")
            row[column] = row[column].replace(word, new_word)
  return data_frame

In [4]:
ft = fasttext.load_model('/content/drive/MyDrive/NLP/cc.fa.300.bin')

In [5]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = word_tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df_train = pd.read_csv('/content/drive/MyDrive/NLP/train.txt', sep="\t", on_bad_lines='skip',header=None)
df_train.columns = ["sent", "mood"]

df_test = pd.read_csv('/content/drive/MyDrive/NLP/test.txt', sep="\t", on_bad_lines='skip',header=None)
df_test.columns = ["sent", "mood"]

df_all = df_train.append(df_test)
df_all = preprocess_data_frame(df_all)
df_train = preprocess_data_frame(df_train)
df_test = preprocess_data_frame(df_test)

In [7]:
all_words = []
for index, row in df_all.iterrows():
  for sent in sent_tokenize(row["sent"]):
    for word in word_tokenize(row["sent"]):
      if word not in all_words: 
        all_words.append(word)

vocab2index = {"":0, "UNK":1}
word_vector_dic = {}
for word in all_words:
  vocab2index[word] = len(vocab2index.keys())
  if word not in word_vector_dic:
    word_vector_dic[word] = ft.get_word_vector(word)

In [21]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = word_tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

df_train['encoded'] = df_train['sent'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))
df_test['encoded'] = df_test['sent'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [22]:
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(df_train["mood"])
y_test = Encoder.fit_transform(df_test["mood"])


X_test = pd.Series(df_test["encoded"].values.tolist())
y_test = np.array(y_test)

X_train, X_val, y_train, y_val = train_test_split(df_train["encoded"], y_train, test_size=0.25, random_state=1)

In [10]:
print(type(y_test))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [23]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [24]:
train_ds = ReviewsDataset(list(X_train), y_train)
valid_ds = ReviewsDataset(list(X_val), y_val)
test_ds = ReviewsDataset(list(X_test), y_test)
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [13]:
del(ft)

In [25]:
def get_emb_matrix(emb_size = 300):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_vector_dic.keys()) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_vector_dic.keys():
        if word in word_vector_dic:
            W[i] = word_vector_dic[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx, vocab_size

In [31]:
pretrained_weights, vocab, vocab2index, vocab_size = get_emb_matrix(emb_size = 300)

In [32]:
print(len(word_vector_dic.keys()))

19141


In [33]:
class LSTM_glove_vecs(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 7)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [34]:
import math
from sklearn.metrics import classification_report,confusion_matrix
def train_model(model, epochs, lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    print(epochs)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x.cuda(), l.cuda())
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred.cuda(), y.cuda())
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 20 == 0:
            print("epoch %d train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (i, sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long().cuda()
        y = y.long().cuda()
        y_hat = model(x.cuda(), l.cuda())
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct = correct + (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        a = mean_squared_error(pred.cpu(), y.unsqueeze(-1).cpu())
        sum_rmse += math.sqrt(mean_squared_error(pred.cpu(), y.unsqueeze(-1).cpu()))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

def test_metrics(model, test_dl):
    model.to(torch.device("cpu"))
    model.eval()
    predictions = []
    labels = []
  
    for x, y, l in test_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        pred = torch.max(y_hat, 1)[1]
        pred = list(pred.cpu().detach().numpy())
        y = list(y.cpu().detach().numpy())
        predictions += pred
        labels += y

    print(classification_report(labels,predictions))

In [30]:
model = LSTM_glove_vecs(vocab_size = vocab_size, embedding_dim = 300, hidden_dim = 50, weights = pretrained_weights)
model.to(device)
train_model(model, epochs=300, lr=0.01)
test_metrics(model,test_dl)

300
epoch 0 train loss 1.899, val loss 1.881, val accuracy 0.279, and val rmse 2.096
epoch 20 train loss 1.787, val loss 1.785, val accuracy 0.273, and val rmse 2.164
epoch 40 train loss 0.848, val loss 1.329, val accuracy 0.557, and val rmse 2.170
epoch 60 train loss 0.375, val loss 1.611, val accuracy 0.575, and val rmse 1.916
epoch 80 train loss 0.233, val loss 1.841, val accuracy 0.588, and val rmse 1.928
epoch 100 train loss 0.156, val loss 1.996, val accuracy 0.585, and val rmse 1.952
epoch 120 train loss 0.121, val loss 2.176, val accuracy 0.587, and val rmse 1.928
epoch 140 train loss 0.111, val loss 2.355, val accuracy 0.589, and val rmse 1.968
epoch 160 train loss 0.085, val loss 2.400, val accuracy 0.582, and val rmse 1.953
epoch 180 train loss 0.062, val loss 2.448, val accuracy 0.586, and val rmse 1.957
epoch 200 train loss 0.057, val loss 2.524, val accuracy 0.584, and val rmse 1.954
epoch 220 train loss 0.067, val loss 2.372, val accuracy 0.587, and val rmse 1.885
epoch 